In [1]:
using RigidBodyDynamics

In [2]:
frame1 = CartesianFrame3D("foo")

RigidBodyDynamics.CartesianFrame3D("foo")

In [5]:
frame1 == frame1

true

In [4]:
@which CartesianFrame3D("foo")

call(::Type{RigidBodyDynamics.CartesianFrame3D}, name::ASCIIString) at /home/rdeits/.julia/v0.4/RigidBodyDynamics/src/frames.jl:12